In [1]:
import sys
import numpy as np

#### Import Unity Agents

In [2]:
from unityagents import UnityEnvironment


In [3]:
env  = UnityEnvironment(file_name="UnityEnv/Banana_Linux/Banana.x86_64")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


#### Training the Brain with deepQLearning

In [20]:
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
env_info = env.reset(train_mode=True)[brain_name]
observtaion_space = env_info.vector_observations[0]
print(observtaion_space)

[0.         1.         0.         0.         0.26121268 0.
 1.         0.         0.         0.19224221 0.         1.
 0.         0.         0.24639925 0.         1.         0.
 0.         0.27691892 0.         1.         0.         0.
 0.18541232 0.         1.         0.         0.         0.24639931
 0.         1.         0.         0.         0.17489611 0.
 0.        ]
